In [1]:
import numpy as np
import pandas as pd

from desdeo_problem.Objective import _ScalarObjective
from desdeo_problem.Variable import variable_builder
from desdeo_problem.Problem import MOProblem
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_
from pygmo import fast_non_dominated_sorting as nds
import plotly.express as ex
from desdeo_problem.Constraint import ScalarConstraint

## Define Objectives and variable builder

In [2]:
def f_1(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return 0.5*term + np.sin(term)

def f_2(x):
    term1 = ((3*x[:, 0] - 2*x[:,1] + 4)**2)/8
    term2 = ((x[:, 0] - x[:,1] + 1)**2)/27
    return term1 + term2 + 15

def f_3(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return (1/(term + 1)) - 1.1 * np.exp(-term)

list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-30, -30],
                             upper_bounds=[30, 30])

## Define objectives

In [3]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)
f3 = _ScalarObjective(name='f3', evaluator=f_3)

In [4]:
from desdeo_problem.Constraint import ScalarConstraint

const_func_1 = lambda x, y: (y[:, 0] - .2) * (y[:, 0] - .4) * (-1)
const_func_2 = lambda x, y: 16.5 - y[:, 1]

# Args: name, number of variables, number of objectives, callable
cons1 = ScalarConstraint("c_1", 2, 2, const_func_1)
cons2 = ScalarConstraint("c_2", 2, 2, const_func_2)

## Define MOProblem

In [5]:
problem = MOProblem(variables=list_vars, objectives=[f1, f2], constraints=[cons2])

# problem = MOProblem(variables=list_vars, objectives=[f1, f2])

evolver = RVEA(problem)

In [6]:
figure = animate_init_(evolver.population.objectives, filename="MOP5.html")

Plot saved as:  MOP5.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [7]:
i = 0
while i<10:
    print('---------------------------')
    i += 1
    evolver.iterate()
    print(evolver.selection_operator.time_penalty_function())
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOP5.html",
        generation=evolver._iteration_counter,
    )

---------------------------
0.1
---------------------------
0.2
---------------------------
0.3
---------------------------
0.4
---------------------------
0.5
---------------------------
0.6
---------------------------
0.7
---------------------------
0.8
---------------------------
0.9
---------------------------
1.0


## MOP-C4 Tanaka from Coello's book

The problem is wrong in the book, look up literature.

In [2]:
def f_1(x):
    return x[:,0]

def f_2(x):
    return x[:,1]

def c_1(x, y):
    a = 0.1
    b = 16
    #a = 0.1 * (x[:,0]**2 + x[:,1]**2 + 5 * x[:,0]*x[:,1])
    #b = 8 * (x[:,0]**2 + x[:,1]**2)
    return -x[:,0]**2 - x[:,1]**2 + 1 + a* np.cos(b*np.arctan(x[:,0]/x[:,1]))

def c_2(x,y):
    return - 0.5 + (x[:,0]-0.5)**2 + (x[:,1]-0.5)**2

In [3]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[0, 0],
                             upper_bounds=[np.pi, np.pi])

In [4]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)

In [5]:
c1 = ScalarConstraint("c1", 2, 2, c_1)
c2 = ScalarConstraint("c2", 2, 2, c_2)

In [6]:
problem = MOProblem(variables=list_vars, objectives=[f1,f2], constraints=[c1,c2])

In [12]:
evolver = RVEA(problem, lattice_resolution=200, alpha=0.1)

In [13]:
figure = animate_init_(evolver.population.objectives, filename="MOPC4.html")

while evolver.continue_evolution():
    evolver.iterate()
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOPC4.html",
        generation=evolver._iteration_counter,
    )

Plot saved as:  MOPC4.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


/home/rishi/.virtualenvs/desdeo-emo/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

divide by zero encountered in true_divide

/home/rishi/.virtualenvs/desdeo-emo/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in true_divide



In [14]:
non_dom = evolver.population.objectives[nds(evolver.population.objectives)[0][0]]

In [15]:
ex.scatter(x=non_dom[:,0], y=non_dom[:,1])

## Viennet 4

In [2]:
def f_1(x):
    return ((x[:,0] - 2) ** 2) / 2 + ((x[:,1] + 1) ** 2) / 13 + 3

def f_2(x):
    return ((x[:,0] + x[:,1] - 3)**2)/175+ ((2*x[:,1]-x[:,0])**2)/17 -13

def f_3(x):
    return ((3*x[:,0]-2*x[:,1]+4)**2)/8 + ((x[:,0]-x[:,1]+1)**2)/27 + 15

In [3]:
def c_1(x,y):
    return -3*x[:,0]+4-x[:,1] - np.finfo(float).eps

def c_2(x,y):
    return x[:,0] + 1 - np.finfo(float).eps

def c_3(x,y):
    return x[:,1]-x[:,0]+ 2-np.finfo(float).eps

In [4]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-4, -4],
                             upper_bounds=[4, 4])

In [5]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)
f3 = _ScalarObjective(name='f3', evaluator=f_3)

In [6]:
c1 = ScalarConstraint("c1", n_decision_vars=2, n_objective_funs=3, evaluator=c_1)
c2 = ScalarConstraint("c2", n_decision_vars=2, n_objective_funs=3, evaluator=c_2)
c3 = ScalarConstraint("c3", n_decision_vars=2, n_objective_funs=3, evaluator=c_3)


In [7]:
problem = MOProblem(variables=list_vars, objectives=[f1,f2, f3], constraints=[c1,c2,c3])

In [8]:
evolver = RVEA(problem, lattice_resolution=50)

In [9]:
figure = animate_init_(evolver.population.objectives, filename="MOPC4.html")
while evolver.continue_evolution():
    evolver.iterate()
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOPC4.html",
        generation=evolver._iteration_counter,
    )

Plot saved as:  MOPC4.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [34]:
non_dom = evolver.population.objectives[nds(evolver.population.objectives)[0][0]]

In [36]:
ex.scatter_3d(x=non_dom[:,0]+6, y=non_dom[:,1], z=non_dom[:,2])

## Using EvoNN

In [8]:
from desdeo_emo.surrogatemodelling.EvoNN import EvoNN
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score

In [9]:
df = pd.read_excel('final bashi project.xlsx', error_bad_lines=False)

col = df.columns
X_col = col[0:12]
y_col = col[12:20]

X = df[X_col].values

In [11]:
models = []
i = 0
for col in y_col:
    print(i)
    model = EvoNN(pop_size=100, activation_function='relu')
    model.fit(df[X_col], df[col])
    y_pred = model.predict(X)
    models.append(model)
    i += 1


0
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
1
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
2
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
3
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
4
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
5
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
6
Plot saved as:  EvoNN.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
7
Plot saved as:  EvoNN.html
View the plo